In [1]:
import nltk
import torch.utils.data as data
import numpy as np
import os
import requests
import time
import torch
nltk.download('punkt')
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import torch.optim as optim
import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alejg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


version 2


In [2]:
training_path_json="D:/datos/annotations_trainval2017/annotations/captions_train2017.json"
training_path_images="D:/datos/train2017/train2017/"
validation_path_json="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/annotations/captions_val2014.json"
validation_path_images="C:/Users/alejg/Documents/proyectos_personales/images_segmentation/coco/images/val2014/"


In [3]:
## TODO #1: Select appropriate values for the Python variables below.
batch_size = 400#400<-          # batch size
vocab_threshold = 4 #<-      # minimum word count threshold
vocab_from_file = False  #<-  # if True, load existing vocab file
embed_size =   256#<-         # dimensionality of image and word embeddings
hidden_size = 512#<-         # number of features in hidden state of the RNN decoder
num_epochs = 10             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

epoch_ini=9

In [4]:
os.makedirs("saved_models", exist_ok=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [5]:
# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

In [6]:
train_dataloader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file,
                             images_path=training_path_images,annotation_path=training_path_json)

loading annotations into memory...
Done (t=0.76s)
creating index...
index created!
[0/591753] Tokenizing captions...
[100000/591753] Tokenizing captions...
[200000/591753] Tokenizing captions...
[300000/591753] Tokenizing captions...
[400000/591753] Tokenizing captions...
[500000/591753] Tokenizing captions...
loading annotations into memory...


  0%|▏                                                                        | 1205/591753 [00:00<00:49, 11930.80it/s]

Done (t=0.75s)
creating index...
index created!
Obtaining caption lengths...


100%|███████████████████████████████████████████████████████████████████████| 591753/591753 [00:46<00:00, 12859.44it/s]


In [7]:
validation_dataloader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=10,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=True,
                                  images_path=validation_path_images,annotation_path=validation_path_json)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  1%|▉                                                                        | 2504/202654 [00:00<00:16, 12450.04it/s]

Done (t=0.29s)
creating index...
index created!
Obtaining caption lengths...


100%|███████████████████████████████████████████████████████████████████████| 202654/202654 [00:16<00:00, 12553.67it/s]


In [8]:
# The size of the vocabulary.
vocab_size = len(train_dataloader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)


In [9]:
encoder.to(device)
decoder.to(device)

DecoderRNN(
  (word_embeddings): Embedding(11543, 256)
  (lstm): LSTM(256, 512, batch_first=True)
  (linear): Linear(in_features=512, out_features=11543, bias=True)
)

In [10]:
vocab_size

11543

In [11]:
embed_size = 256#<-
hidden_size = 512#<-
vocab_size = 11543#9999

encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)



In [12]:
print(f"epoch_ini= {epoch_ini}")
if epoch_ini != 0:
    # Load pretrained models
    
    encoder.load_state_dict(torch.load("saved_models/v3encoder_%d.pkl" % epoch_ini))
    decoder.load_state_dict(torch.load("saved_models/v3decoder_%d.pkl" % epoch_ini))
    epoch_ini=epoch_ini+1
    print("loaded")

epoch_ini= 9
loaded


In [ ]:
encoder.to(device)
decoder.to(device)

In [ ]:
# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

In [ ]:
#params = list(decoder.parameters())  + list(encoder.parameters())
params =list(filter(lambda p: p.requires_grad,decoder.parameters()))+list(filter(lambda p: p.requires_grad,encoder.parameters()))

# TODO #4: Define the optimizer.
optimizer = optim.Adam(params, lr=0.001, betas=(0.9, 0.999))#optim.SGD(net.parameters(), lr=0.01, momentum=0.9)



# Set the total number of training steps per epoch.
total_step = math.ceil(len(train_dataloader.dataset.caption_lengths) / train_dataloader.batch_sampler.batch_size)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
#tensorboard --logdir=C:/Users/alejg/Documents/tfm/image_captioning_backend/train/tensroboar



In [ ]:
## Open the training log file.
#f = open(log_file, 'w')
#
#for epoch in range(epoch_ini, num_epochs):
#    
#    for i_step in range(1, total_step+1):
#        # Randomly sample a caption length, and sample indices with that length.
#        indices = train_dataloader.dataset.get_train_indices()
#        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
#        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
#        train_dataloader.batch_sampler.sampler = new_sampler
#        # Obtain the batch.
#        images, captions = next(iter(train_dataloader))
#
#        # Move batch of images and captions to GPU if CUDA is available.
#        images = images.to(device)
#        captions = captions.to(device)
#        
#        # Zero the gradients.
#        decoder.zero_grad()
#        encoder.zero_grad()
#        
#        # Pass the inputs through the CNN-RNN model.
#        features = encoder(images)
#        outputs = decoder(features, captions)
#        
#        # Calculate the batch loss.
#        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
#        
#        # Backward pass.
#        loss.backward()
#        
#        # Update the parameters in the optimizer.
#        optimizer.step()
#            
#        # Get training statistics.
#        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
#        
#        # Print training statistics (on same line).
#        print('\r' + stats, end="")
#        sys.stdout.flush()
#        
#        # Print training statistics to file.
#        f.write(stats + '\n')
#        f.flush()
#        
#        # Print training statistics (on different line).
#        if i_step % print_every == 0:
#            print('\r' + stats)
#            
#    # Save the weights.
#    if epoch % save_every == 0:
#        torch.save(decoder.state_dict(), "saved_models/v3decoder_%d.pkl" % epoch)
#        torch.save(encoder.state_dict(), "saved_models/v3encoder_%d.pkl" % epoch)
#
## Close the training log file.
#f.close()

In [ ]:
len(train_dataloader.dataset.caption_lengths)

In [ ]:
len(validation_dataloader.dataset.caption_lengths)

In [ ]:
def mi_validation(decoder,encoder, loader, criterion):
    test_loss = 0
    total_step = math.ceil(len(loader.dataset.caption_lengths) / loader.batch_sampler.batch_size)
    vocab_size = len(loader.dataset.vocab)
    perplexityAcum=0
    perplexityVec=[]
    for i_step in tqdm.tqdm(range(1, total_step+1)):
        images, captions = next(iter(loader))
         # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        perplexity=np.exp(loss.item())
        perplexityVec.append(perplexity)
        perplexityAcum +=perplexity
        test_loss += loss   
        
    return test_loss,perplexityAcum,np.mean(perplexityVec)

In [ ]:
writer = SummaryWriter('tensroboar')
# Open the training log file.
f = open(log_file, 'w')

for epoch in range(epoch_ini, num_epochs):
    decoder.train()
    encoder.train()
    for i_step in tqdm.tqdm(range(1, total_step+1)):
        # Randomly sample a caption length, and sample indices with that length.
        indices = train_dataloader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        train_dataloader.batch_sampler.sampler = new_sampler
        # Obtain the batch.
        images, captions = next(iter(train_dataloader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    decoder.eval()
    encoder.eval()
    with torch.no_grad():
        train_loss,train_perplexityAcum,train_perplexityMean = mi_validation(decoder,encoder, train_dataloader, criterion)
        
    totSteptrain = math.ceil(len(train_dataloader.dataset.caption_lengths) / train_dataloader.batch_sampler.batch_size)
    losstrain=train_loss/totSteptrain
    print(f"loss training {losstrain}")
    writer.add_scalar('loss_training', losstrain, epoch)
    print(f"perplexityAcum training {train_perplexityAcum}")
    writer.add_scalar('train_perplexityAcum', train_perplexityAcum, epoch)
    print(f"perplexityMean training {train_perplexityMean}")
    writer.add_scalar('train_perplexityMean', train_perplexityMean, epoch)
    
    
    decoder.eval()
    encoder.eval()
    with torch.no_grad():
        test_loss,test_perplexityAcum,test_perplexityMean= mi_validation(decoder,encoder, validation_dataloader, criterion)
    
    totStepValid = math.ceil(len(validation_dataloader.dataset.caption_lengths) / validation_dataloader.batch_sampler.batch_size)
    losstest=test_loss/totStepValid
    print(f"loss validation {losstest}")
    writer.add_scalar('loss_validation', losstest, epoch)
    print(f"perplexityAcum validation {test_perplexityAcum}")
    writer.add_scalar('test_perplexityAcum', test_perplexityAcum, epoch)
    print(f"perplexityMean validation {test_perplexityMean}")
    writer.add_scalar('test_perplexityMean', test_perplexityMean, epoch)
    
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), "saved_models/v3decoder_%d.pkl" % epoch)
        torch.save(encoder.state_dict(), "saved_models/v3encoder_%d.pkl" % epoch)

# Close the training log file.
f.close()

In [ ]:
epoch